In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
pd.options.display.max_columns = None

In [31]:
# df = pd.read_csv("./all/train.csv")

df = pd.read_csv("./all/test.csv")

# df_test = pd.read_csv("./all/test.csv")
# df = pd.concat([df_test, df])

In [32]:
values = {'PoolQC': 'noPool', 'MiscFeature': 'none', 'Alley': 'noAlley', 'Fence': 'noFence', 
          'FireplaceQu': 'noFireplace', 'GarageCond': 'noGarage', 'GarageType': 'noGarage', 'GarageYrBlt': 'noGarage',
          'GarageFinish': 'noGarage', 'GarageQual': 'noGarage', 'GarageCars': 0, 'GarageArea': 0,
          'BsmtExposure': 'noBasement',  'BsmtFinType2': 'noBasement', 'BsmtFinType1': 'noBasement',
          'BsmtCond': 'noBasement', 'BsmtQual': 'noBasement', 'BsmtFinSF2': 0, 'BsmtFinSF1': 0, 
          'BsmtUnfSF': 0, 'TotalBsmtSF': 0, 'BsmtFullBath': 0, 'BsmtHalfBath': 0, 'MasVnrArea': 0, 'MasVnrType': 'None'}

df = df.fillna(value=values)

cat_to_imputation = ['Exterior1st', 'Exterior2nd', 'MSZoning', 'SaleType', 'KitchenQual', 
                     'Functional', 'Utilities', 'Electrical','MasVnrType']

for col in cat_to_imputation:
    df[col] = df[col].fillna(df[col].value_counts().index[0])



In [33]:
# sort out basement data

base_cond = (df.BsmtCond == "noBasement") & (df.BsmtFinSF1 > 0)
df.loc[base_cond, "BsmtCond"] = "TA"

base_exp = (df.BsmtExposure == "noBasement") & (df.BsmtUnfSF > 0)
df.loc[base_exp, "BsmtExposure"] = "No"

base_fin2 = (df.BsmtFinType2 == "noBasement") & (df.BsmtFinSF2 > 0)
df.loc[base_fin2, "BsmtFinType2"] = "GLQ"

base_qual = (df.BsmtQual == "noBasement") & (df.BsmtUnfSF > 0)
df.loc[base_qual, "BsmtQual"] = "Gd"

In [34]:
# sort out garage data - SAM i just changed df_all to df?

rm_garage_type = (df.GarageType != 'noGarage') & ((df.GarageCars == 0.0) & (df.GarageArea == 0.0))
df.loc[rm_garage_type, 'GarageType'] = 'noGarage'

fill_garage_modes = (df.GarageType == 'Detchd') & ((df.GarageCars != 0.0) & (df.GarageArea != 0.0))
df_detached_garage = df[df.GarageType == 'Detchd']
df.loc[fill_garage_modes, 'GarageFinish'] = df_detached_garage.GarageFinish.value_counts().index[0]
df.loc[fill_garage_modes, 'GarageCond'] = df_detached_garage.GarageCond.value_counts().index[0]
df.loc[fill_garage_modes, 'GarageQual'] = df_detached_garage.GarageQual.value_counts().index[0]

In [35]:
df.isnull().sum().sort_values(ascending=False).head(10)

LotFrontage      227
SaleCondition      0
Foundation         0
RoofMatl           0
Exterior1st        0
Exterior2nd        0
MasVnrType         0
MasVnrArea         0
ExterQual          0
ExterCond          0
dtype: int64

In [29]:
# just LotFrontage to deal with 
# could just leave this variable out?

In [36]:
df.to_csv("test_cleaned.csv")

In [39]:
check = (df.MasVnrArea == 0) & (df.MasVnrType != "None")
df[check]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
859,2320,20,RL,64.0,7697,Pave,noAlley,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,1Story,7,6,2007,2007,Gable,CompShg,VinylSd,VinylSd,BrkFace,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,1246.0,1246.0,GasA,Ex,Y,SBrkr,1258,0,0,1258,0.0,0.0,2,0,3,1,Gd,6,Typ,0,noFireplace,Attchd,2007,RFn,2.0,462.0,TA,TA,Y,0,0,0,0,0,0,noPool,noFence,none,0,8,2007,New,Partial


In [40]:
check = (df.MasVnrArea != 0) & (df.MasVnrType == "None")
df[check]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
209,1670,20,RL,102.0,13514,Pave,noAlley,IR1,Lvl,AllPub,Corner,Gtl,NridgHt,Norm,Norm,1Fam,1Story,9,5,2008,2008,Hip,CompShg,VinylSd,VinylSd,None,285.0,Ex,TA,PConc,Ex,TA,No,GLQ,1142.0,Unf,0.0,632.0,1774.0,GasA,Ex,Y,SBrkr,1808,0,0,1808,1.0,0.0,2,0,3,1,Ex,7,Typ,1,Gd,Attchd,2008,Fin,3.0,850.0,TA,TA,Y,200,26,0,0,0,0,noPool,noFence,none,0,3,2009,WD,Normal
992,2453,20,RM,52.0,8626,Pave,noAlley,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,4,6,1956,1956,Gable,CompShg,MetalSd,MetalSd,None,1.0,TA,TA,CBlock,noBasement,noBasement,noBasement,noBasement,0.0,noBasement,0.0,0.0,0.0,GasA,Gd,Y,SBrkr,968,0,0,968,0.0,0.0,1,0,2,1,TA,5,Typ,0,noFireplace,Attchd,1956,Unf,1.0,331.0,Fa,TA,Y,0,0,0,0,0,0,noPool,noFence,none,0,5,2007,WD,Normal
1150,2611,20,RL,124.0,27697,Pave,noAlley,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,4,3,1961,1961,Shed,CompShg,Plywood,Plywood,None,198.0,TA,TA,CBlock,TA,TA,No,BLQ,811.0,Unf,0.0,585.0,1396.0,GasA,TA,N,SBrkr,1608,0,0,1608,0.0,0.0,1,0,3,1,TA,6,Typ,0,noFireplace,Attchd,1961,Unf,1.0,444.0,TA,Fa,Y,152,38,0,0,0,0,noPool,noFence,none,0,11,2007,COD,Abnorml


In [45]:
check_base_sum = df[["TotalBsmtSF", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF"]]
check_base_sum["check_total"] = check_base_sum["BsmtFinSF1"] + check_base_sum["BsmtFinSF2"] + check_base_sum["BsmtUnfSF"] - check_base_sum["TotalBsmtSF"]

check_base_sum[check_base_sum["check_total"] != 0]
# check_base_sum.sort_values("check_total")

/Users/fcrouch/miniconda3/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,TotalBsmtSF,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,check_total
